In [ ]:
!pip install --upgrade tqdm
!pip install keras_metrics

     |████████████████████████████████| 71kB 2.9MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models import KeyedVectors
import multiprocessing
from sklearn import utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import keras
import keras_metrics
from keras.callbacks import TensorBoard
import tensorflow as tf

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlinBackend.figure_format = 'retina'

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
# %cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/selected_foods.csv')
df.head()

,sentence,label
0,i love this water it does not taste good unles...,4
1,we have been looking for pesto that tastes as ...,4
2,i make own sausages back in ireland and this i...,4
3,anything that says enriched in the ingredients...,1
4,amazon normally does a fantastic job getting p...,2


In [ ]:
df.label = df.label-1
df.head()

,sentence,label
0,i love this water it does not taste good unles...,3
1,we have been looking for pesto that tastes as ...,3
2,i make own sausages back in ireland and this i...,3
3,anything that says enriched in the ingredients...,0
4,amazon normally does a fantastic job getting p...,1


In [ ]:
x = df.sentence
y = df.label
SEED = 666
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.2, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [ ]:
y_train = to_categorical(y_train)
y_validation = to_categorical(y_validation)

In [ ]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
x_train_seq = pad_sequences(sequences, maxlen=40)

sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=40)

sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=40)
y_test = to_categorical(y_test)

In [ ]:
tensorboard = TensorBoard(log_dir="./logs/stacked_bigru")
keras_callbacks = [tensorboard]

In [ ]:
%%time
#GRU bidirectional
num = 2 #define how many stackings

#forward layers
rnn_cells_forward = [tf.keras.layers.GRUCell(100, dropout=0.2) for _ in range(num)]
stacked_forward = tf.keras.layers.StackedRNNCells(rnn_cells_forward)

#backward layers
rnn_cells_backward = [tf.keras.layers.GRUCell(100, dropout=0.2) for _ in range(num)]
stacked_backward = tf.keras.layers.StackedRNNCells(rnn_cells_backward)

forward_layer = tf.keras.layers.RNN(stacked_forward, return_sequences=True)
backward_layer = tf.keras.layers.RNN(stacked_backward, return_sequences=True, go_backwards=True)

model = tf.keras.Sequential() 
# e = tf.keras.layers.Embedding(100000, 200, weights=[embedding_matrix], input_length=40
#               , trainable=True)
e = tf.keras.layers.Embedding(100000, 200,  input_length=40)
model.add(e)
model.add(tf.keras.layers.Bidirectional(forward_layer, backward_layer= backward_layer))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', keras_metrics.precision(), 
                           keras_metrics.recall(), keras_metrics.f1_score()])
print(model.summary())
history = model.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=4, batch_size=64, callbacks=keras_callbacks)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 200)           20000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 200)           302400    
_________________________________________________________________
flatten (Flatten)            (None, 8000)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                512064    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 20,814,789
Trainable params: 20,814,789
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/4
Instructions for updating:
`inputs` is now automatically inferred
Instructions for updating:
Th

In [ ]:
model.evaluate(x_test_seq, y_test) #new tab

157/157 [==============================] - 5s 34ms/step - loss: 1.6796 - accuracy: 0.5178 - precision: 0.6120 - recall: 0.6078 - f1_score: 0.6096


[1.6795982122421265,
 0.517799973487854,
 0.6120175123214722,
 0.6077976226806641,
 0.6095637083053589]

In [ ]:
model.evaluate(x_test_seq, y_test)

157/157 [==============================] - 5s 34ms/step - loss: 1.6796 - accuracy: 0.5178 - precision: 0.6120 - recall: 0.6078 - f1_score: 0.6096


[1.6795982122421265,
 0.517799973487854,
 0.6120175123214722,
 0.6077976226806641,
 0.6095637083053589]

In [ ]:
%%time
#LSTM bidirectional
num = 2 #define how many stackings

#forward layers
rnn_cells_forward = [tf.keras.layers.LSTMCell(100, dropout=0.2) for _ in range(num)]
stacked_forward = tf.keras.layers.StackedRNNCells(rnn_cells_forward)

#backward layers
rnn_cells_backward = [tf.keras.layers.LSTMCell(100, dropout=0.2) for _ in range(num)]
stacked_backward = tf.keras.layers.StackedRNNCells(rnn_cells_backward)

forward_layer = tf.keras.layers.RNN(stacked_forward, return_sequences=True)
backward_layer = tf.keras.layers.RNN(stacked_backward, return_sequences=True, go_backwards=True)

model = tf.keras.Sequential() 
e = tf.keras.layers.Embedding(100000, 200, input_length=40)
model.add(e)
model.add(tf.keras.layers.Bidirectional(forward_layer, backward_layer= backward_layer))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', keras_metrics.precision(), 
                           keras_metrics.recall(), keras_metrics.f1_score()])
print(model.summary())
history = model.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=4, batch_size=64, callbacks=keras_callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           20000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40, 200)           401600    
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                512064    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 20,913,989
Trainable params: 20,913,989
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/4
625/625 [==============================] - 175s 281ms/step - loss: 1.2837 - accuracy: 0.4421 

In [ ]:
model.evaluate(x_test_seq, y_test)

157/157 [==============================] - 4s 28ms/step - loss: 1.5546 - accuracy: 0.5282 - precision: 0.6219 - recall: 0.6274 - f1_score: 0.6244


[1.5546308755874634,
 0.5281999707221985,
 0.6219314336776733,
 0.6273571848869324,
 0.6244449615478516]

In [ ]:
model.evaluate(x_test_seq, y_test)

157/157 [==============================] - 4s 27ms/step - loss: 1.5546 - accuracy: 0.5282 - precision: 0.6219 - recall: 0.6274 - f1_score: 0.6244


[1.5546308755874634,
 0.5281999707221985,
 0.6219314336776733,
 0.6273571848869324,
 0.6244449615478516]